In [8]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import yt_dlp
import re
import os

In [1]:
# URL de la lista
url_playlist = "https://open.spotify.com/playlist/37i9dQZF1DX9235ICuUcIM"
# https://open.spotify.com/playlist/6IuU86UaJm6fxeN6PeMPCT (sad playlist)

In [10]:
# Configuración de credenciales de cliente de Spotify
client_id = 'YOUR CLIENT ID'
client_secret = 'YOUR CLIENT PASSWORD'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


In [11]:
# Obtener tracks de una playlist de Spotify
def obtener_tracks_playlist(playlist_id):
    tracks = []
    offset = 0
    # Iteraremos sobre cada una de las paginas (items) de la lista de reproducción
    while True:
        results = sp.playlist_tracks(playlist_id, offset=offset)
        items = results.get('items', [])
        if not items:
            break
        for item in items:
            track = item.get('track')
            if track:
                tracks.append((track.get('name', ''), track.get('artists', [{}])[0].get('name', '')))
        offset += len(items)
    return tracks


In [15]:
# descargar las listas usanto convertidor YT a mp3
def download_audio(link, output_folder, song_name, artist):
    # Usamos el nombre de la canción y el artista para formar el nombre del archivo
    file_name = f"{song_name} - {artist}"
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'outtmpl': os.path.join(output_folder, file_name),
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info_dict = ydl.extract_info(link, download=True)
        video_title = info_dict['title']
        print(f"Descargada: {file_name}")

In [2]:
playlist_id = re.search(r'playlist\/(\w{22})', url_playlist).group(1)
output_folder = r'C:\Users\...'
tracks = obtener_tracks_playlist(playlist_id)
total_tracks = len(tracks)
tracks_downloaded = 0

for track in tracks:
    song_name, artist = track
    search_query = f"{song_name} {artist} audio"
    link = f"ytsearch1:{search_query}"
    download_audio(link, output_folder, song_name, artist)
    tracks_downloaded += 1
    print(print(f"Se han descargado {tracks_downloaded} canciones ({100 * tracks_downloaded / total_tracks:.2f}%)"))

NameError: name 're' is not defined